**Import the necesary libraries for data reading / manipulation, directory changes.**
<br>
[Pandas](https://pandas.pydata.org/)
<br>
[OS](https://docs.python.org/3/library/os.html)

In [1]:
import pandas as pd 
import os
pd.set_option('display.max_columns',None)

**You can change path if you want**

In [2]:
#path = os.getcwd()
#os.chdir(os.path.join(path)+'/Downloads')

**Read data from QSS repo (about houses)**

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/QSS-Analytics/Datasets/master/900_houses.csv')

In [4]:
df.head(3)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19
0,Yeni tikili,16 / 19,130 m²,3,var,235 000AZN,mülkiyyətçi,7101,09 Avqust 2019,Cavanşir körpüsü \n,Nəsimi r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Yeni tikili,4 / 21,164 m²,3,var,237 000AZN,mülkiyyətçi,290,Bugün 10,Şah İsmayıl Xətai m.\n,Xətai r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yeni tikili,3 / 12,80 m²,2,var,55 400AZN,mülkiyyətçi,8426,11 Avqust 2019,Abşeron r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Change column names**

In [5]:
df.columns = ['nov','mertebe','sahe','otaq','kupca','qiymet','elani_yerlesdiren','baxis_sayi',
              'tarix'] + ["Adres_" + str(i) for i in range(1,11)]

In [6]:
df.head(3)

,nov,mertebe,sahe,otaq,kupca,qiymet,elani_yerlesdiren,baxis_sayi,tarix,Adres_1,Adres_2,Adres_3,Adres_4,Adres_5,Adres_6,Adres_7,Adres_8,Adres_9,Adres_10
0,Yeni tikili,16 / 19,130 m²,3,var,235 000AZN,mülkiyyətçi,7101,09 Avqust 2019,Cavanşir körpüsü \n,Nəsimi r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Yeni tikili,4 / 21,164 m²,3,var,237 000AZN,mülkiyyətçi,290,Bugün 10,Şah İsmayıl Xətai m.\n,Xətai r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yeni tikili,3 / 12,80 m²,2,var,55 400AZN,mülkiyyətçi,8426,11 Avqust 2019,Abşeron r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Split mertebe into 2 columns based on symbol '/'**

In [7]:
df = df.join(df['mertebe'].str.split('/', 1, expand=True).rename(
    columns = {0:'min_mertebe', 1:'max_mertebe'})).drop(['mertebe','tarix'],1)

In [8]:
df.head(2)

,nov,sahe,otaq,kupca,qiymet,elani_yerlesdiren,baxis_sayi,Adres_1,Adres_2,Adres_3,Adres_4,Adres_5,Adres_6,Adres_7,Adres_8,Adres_9,Adres_10,min_mertebe,max_mertebe
0,Yeni tikili,130 m²,3,var,235 000AZN,mülkiyyətçi,7101,Cavanşir körpüsü \n,Nəsimi r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,19
1,Yeni tikili,164 m²,3,var,237 000AZN,mülkiyyətçi,290,Şah İsmayıl Xətai m.\n,Xətai r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,21


**Clean data from special symbols**

In [9]:
df['sahe'], df['qiymet'] = df['sahe'].str.replace('m²', '') , df['qiymet'].str.replace('AZN| ', '') 

In [10]:
df.head(5)

,nov,sahe,otaq,kupca,qiymet,elani_yerlesdiren,baxis_sayi,Adres_1,Adres_2,Adres_3,Adres_4,Adres_5,Adres_6,Adres_7,Adres_8,Adres_9,Adres_10,min_mertebe,max_mertebe
0,Yeni tikili,130,3,var,235000,mülkiyyətçi,7101,Cavanşir körpüsü \n,Nəsimi r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,19
1,Yeni tikili,164,3,var,237000,mülkiyyətçi,290,Şah İsmayıl Xətai m.\n,Xətai r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,21
2,Yeni tikili,80,2,var,55400,mülkiyyətçi,8426,Abşeron r.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,12
3,Yeni tikili,115,3,var,210000,mülkiyyətçi,9706,İnşaatçılar m.\n,Yasamal r.\n,Yasamal q.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,19
4,Köhnə tikili,60,2,var,82000,vasitəçi (agent),131,Sabunçu r.\n,Bakıxanov q.\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,9


**Simple function for applying it to multiple columns**

In [11]:
def clean(data):
    data=data.str.strip()
    return data

In [12]:
df = df[list(df.columns)].apply(clean, axis=1)

In [13]:
df.head(2)

,nov,sahe,otaq,kupca,qiymet,elani_yerlesdiren,baxis_sayi,Adres_1,Adres_2,Adres_3,Adres_4,Adres_5,Adres_6,Adres_7,Adres_8,Adres_9,Adres_10,min_mertebe,max_mertebe
0,Yeni tikili,130,3,var,235000,mülkiyyətçi,7101,Cavanşir körpüsü,Nəsimi r.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,19
1,Yeni tikili,164,3,var,237000,mülkiyyətçi,290,Şah İsmayıl Xətai m.,Xətai r.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,21


**Create ID column for data melting and reshaping into one-hot encoding (dummy)**

In [14]:
df['ID'] = df.index
df.head(5)

,nov,sahe,otaq,kupca,qiymet,elani_yerlesdiren,baxis_sayi,Adres_1,Adres_2,Adres_3,Adres_4,Adres_5,Adres_6,Adres_7,Adres_8,Adres_9,Adres_10,min_mertebe,max_mertebe,ID
0,Yeni tikili,130,3,var,235000,mülkiyyətçi,7101,Cavanşir körpüsü,Nəsimi r.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,19,0
1,Yeni tikili,164,3,var,237000,mülkiyyətçi,290,Şah İsmayıl Xətai m.,Xətai r.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,21,1
2,Yeni tikili,80,2,var,55400,mülkiyyətçi,8426,Abşeron r.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,12,2
3,Yeni tikili,115,3,var,210000,mülkiyyətçi,9706,İnşaatçılar m.,Yasamal r.,Yasamal q.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,19,3
4,Köhnə tikili,60,2,var,82000,vasitəçi (agent),131,Sabunçu r.,Bakıxanov q.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,9,4


**Melt and keep ID**

In [15]:
df2 = df[['ID']+["Adres_" + str(i) for i in range(1,11)]].melt(id_vars=[('ID')])
df2.head(10)

,ID,variable,value
0,0,Adres_1,Cavanşir körpüsü
1,1,Adres_1,Şah İsmayıl Xətai m.
2,2,Adres_1,Abşeron r.
3,3,Adres_1,İnşaatçılar m.
4,4,Adres_1,Sabunçu r.
5,5,Adres_1,Azadlıq Prospekti m.
6,6,Adres_1,Dövlət Statistika Komitəsi
7,7,Adres_1,Yasamal r.
8,8,Adres_1,Rəssamlıq Akademiyası
9,9,Adres_1,Nərimanov r.


**Reshape it**

In [16]:
df2 = df2.pivot_table(index=['ID'], columns=['value'], aggfunc=len).fillna(0).astype('int8')
df2.head(2)

variable                                            \
value 1-ci mikrorayon q. 2-ci mikrorayon q. 20 Yanvar m. 28 May m.   
ID                                                                   
0                      0                  0            0         0   
1                      0                  0            0         0   

                                                                \
value 3-cü mikrorayon q. 4-cü mikrorayon q. 5-ci mikrorayon q.   
ID                                                               
0                      0                  0                  0   
1                      0                  0                  0   

                                                              \
value 6-cı mikrorayon q. 7-ci mikrorayon q. 8-ci kilometr q.   
ID                                                             
0                      0                  0                0   
1                      0                  0                0   

                                                                      \
value 8-ci mikrorayon q. 9-cu mikrorayon q. Abşeron r. Avtovağzal m.   
ID                                                                     
0                      0                  0          0             0   
1                      0                  0          0             0   

                                                                               \
value Axundov bağı Ayna Sultanova heykəli Azadlıq Prospekti m. Azneft meydanı   
ID                                                                              
0                0                      0                    0              0   
1                0                      0                    0              0   

                                                            \
value Azərbaycan Dillər Universiteti Azərbaycan kinoteatrı   
ID                                                           
0                                  0                     0   
1                                  0                     0   

                                                                       \
value Azərbaycan turizm institutu Ağ şəhər Badamdar q. Bakmil m. Bakı   
ID                                                                      
0                               0        0           0         0    0   
1                               0        0           0         0    0   

                                                        \
value Bakı Asiya Universiteti Bakı Dövlət Universiteti   
ID                                                       
0                           0                        0   
1                           0                        0   

                                                                               \
value Bakı Musiqi Akademiyası Bakı Slavyan Universiteti Bakıxanov q. Bayıl q.   
ID                                                                              
0                           0                         0            0        0   
1                           0                         0            0        0   

                                                                              \
value Beşmərtəbə Biləcəri q. Binəqədi r. Cavanşir körpüsü Dostluq kinoteatrı   
ID                                                                             
0              0           0           0                1                  0   
1              0           0           0                0                  0   

                                                                            \
value Dövlət Statistika Komitəsi Dövlət İdarəçilik Akademiyası Dərnəgül m.   
ID                                                                           
0                              0                             0           0   
1                              0                             0           0   

                                                                       \
value E

**Drop indices**

In [17]:
df2.columns = df2.columns.get_level_values(1)
df2 = df2.reset_index(drop=True)
df2.head(3)

value,1-ci mikrorayon q.,2-ci mikrorayon q.,20 Yanvar m.,28 May m.,3-cü mikrorayon q.,4-cü mikrorayon q.,5-ci mikrorayon q.,6-cı mikrorayon q.,7-ci mikrorayon q.,8-ci kilometr q.,8-ci mikrorayon q.,9-cu mikrorayon q.,Abşeron r.,Avtovağzal m.,Axundov bağı,Ayna Sultanova heykəli,Azadlıq Prospekti m.,Azneft meydanı,Azərbaycan Dillər Universiteti,Azərbaycan kinoteatrı,Azərbaycan turizm institutu,Ağ şəhər,Badamdar q.,Bakmil m.,Bakı,Bakı Asiya Universiteti,Bakı Dövlət Universiteti,Bakı Musiqi Akademiyası,Bakı Slavyan Universiteti,Bakıxanov q.,Bayıl q.,Beşmərtəbə,Biləcəri q.,Binəqədi r.,Cavanşir körpüsü,Dostluq kinoteatrı,Dövlət Statistika Komitəsi,Dövlət İdarəçilik Akademiyası,Dərnəgül m.,Elmlər Akademiyası m.,Fontanlar bağı,Gənclik m.,Gəncə,Hövsan q.,Hüseyn Cavid parkı,Həzi Aslanov m.,Həzi Aslanov q.,Koala parkı,Koroğlu m.,Kubinka q.,Lökbatan q.,M.Hüseynzadə parkı,M.Ə.Sabir parkı,Malokan bağı,Masazır q.,Massiv D q.,Memar Əcəmi m.,Memarlıq və İnşaat Universiteti,Milli Konservatoriya,Mingəçevir,Montin bazarı,Mərkəzi Univermaq,Neapol dairəsi,Neft Akademiyası,Neftçi bazası,Neftçilər m.,Nizami kinoteatrı,Nizami m.,Nizami r.,Nəriman Nərimanov m.,Nəriman Nərimanov parkı,Nərimanov heykəli,Nərimanov r.,Nəsimi bazarı,Nəsimi m.,Nəsimi r.,Park Zorge,Pedaqoji Universiteti,Port Baku,Prezident parkı,Qara Qarayev m.,Qaradağ r.,Qaraçuxur q.,Qubernator parkı,Qış parkı,Respublika stadionu,Rusiya səfirliyi,Rəssamlıq Akademiyası,Sabunçu q.,Sabunçu r.,Sahil bağı,Sahil m.,Sevil Qazıyeva parkı,Sirk,Sovetski,Space TV,Sumqayıt,Suraxanı r.,Səbail r.,Səməd Vurğun parkı,TQDK,Texniki Universiteti,Tibb Universiteti,Təhsil Nazirliyi,Ukrayna dairəsi,Xalqlar Dostluğu m.,Xalça Muzeyi,Xırdalan,Xətai r.,Yasamal bazarı,Yasamal q.,Yasamal r.,Yeni Günəşli q.,Yeni Suraxanı q.,Yeni Yasamal q.,Zabitlər parkı,Zabrat q.,Zoopark,Zərifə Əliyeva adına park,İdman kompleksi,İncəsənət və Mədəniyyət Un.,İnşaatçılar m.,İqsadiyyat Universiteti,İzmir parkı,İçəri Şəhər,İçəri Şəhər m.,Şah İsmayıl Xətai m.,Şirvan,Şəhidlər xiyabanı,Şəlalə parkı,Şərq bazarı,Əhmədli m.,Əhmədli q.
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
len(df2.columns)

133

**Join dummies with full dataset**

In [19]:
df[['nov','sahe', 'otaq', 'kupca','qiymet','elani_yerlesdiren','baxis_sayi','min_mertebe','max_mertebe']].join(df2).head(5)

,nov,sahe,otaq,kupca,qiymet,elani_yerlesdiren,baxis_sayi,min_mertebe,max_mertebe,1-ci mikrorayon q.,2-ci mikrorayon q.,20 Yanvar m.,28 May m.,3-cü mikrorayon q.,4-cü mikrorayon q.,5-ci mikrorayon q.,6-cı mikrorayon q.,7-ci mikrorayon q.,8-ci kilometr q.,8-ci mikrorayon q.,9-cu mikrorayon q.,Abşeron r.,Avtovağzal m.,Axundov bağı,Ayna Sultanova heykəli,Azadlıq Prospekti m.,Azneft meydanı,Azərbaycan Dillər Universiteti,Azərbaycan kinoteatrı,Azərbaycan turizm institutu,Ağ şəhər,Badamdar q.,Bakmil m.,Bakı,Bakı Asiya Universiteti,Bakı Dövlət Universiteti,Bakı Musiqi Akademiyası,Bakı Slavyan Universiteti,Bakıxanov q.,Bayıl q.,Beşmərtəbə,Biləcəri q.,Binəqədi r.,Cavanşir körpüsü,Dostluq kinoteatrı,Dövlət Statistika Komitəsi,Dövlət İdarəçilik Akademiyası,Dərnəgül m.,Elmlər Akademiyası m.,Fontanlar bağı,Gənclik m.,Gəncə,Hövsan q.,Hüseyn Cavid parkı,Həzi Aslanov m.,Həzi Aslanov q.,Koala parkı,Koroğlu m.,Kubinka q.,Lökbatan q.,M.Hüseynzadə parkı,M.Ə.Sabir parkı,Malokan bağı,Masazır q.,Massiv D q.,Memar Əcəmi m.,Memarlıq və İnşaat Universiteti,Milli Konservatoriya,Mingəçevir,Montin bazarı,Mərkəzi Univermaq,Neapol dairəsi,Neft Akademiyası,Neftçi bazası,Neftçilər m.,Nizami kinoteatrı,Nizami m.,Nizami r.,Nəriman Nərimanov m.,Nəriman Nərimanov parkı,Nərimanov heykəli,Nərimanov r.,Nəsimi bazarı,Nəsimi m.,Nəsimi r.,Park Zorge,Pedaqoji Universiteti,Port Baku,Prezident parkı,Qara Qarayev m.,Qaradağ r.,Qaraçuxur q.,Qubernator parkı,Qış parkı,Respublika stadionu,Rusiya səfirliyi,Rəssamlıq Akademiyası,Sabunçu q.,Sabunçu r.,Sahil bağı,Sahil m.,Sevil Qazıyeva parkı,Sirk,Sovetski,Space TV,Sumqayıt,Suraxanı r.,Səbail r.,Səməd Vurğun parkı,TQDK,Texniki Universiteti,Tibb Universiteti,Təhsil Nazirliyi,Ukrayna dairəsi,Xalqlar Dostluğu m.,Xalça Muzeyi,Xırdalan,Xətai r.,Yasamal bazarı,Yasamal q.,Yasamal r.,Yeni Günəşli q.,Yeni Suraxanı q.,Yeni Yasamal q.,Zabitlər parkı,Zabrat q.,Zoopark,Zərifə Əliyeva adına park,İdman kompleksi,İncəsənət və Mədəniyyət Un.,İnşaatçılar m.,İqsadiyyat Universiteti,İzmir parkı,İçəri Şəhər,İçəri Şəhər m.,Şah İsmayıl Xətai m.,Şirvan,Şəhidlər xiyabanı,Şəlalə parkı,Şərq bazarı,Əhmədli m.,Əhmədli q.
0,Yeni tikili,130,3,var,235000,mülkiyyətçi,7101,16,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Yeni tikili,164,3,var,237000,mülkiyyətçi,290,4,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Yeni tikili,80,2,var,55400,mülkiyyətçi,8426,3,12,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Yeni tikili,115,3,var,210000,mülkiyyətçi,9706,17,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Köhnə tikili,60,2,var,82000,vasitəçi (agent),131,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Next time we will continue data processing for build a ML model**